In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, RecursiveMap, Tuple, Value, Structural } from "recursive-set";
import { DFA, NFA, State, Char, DFAState } from "./01-NFA-2-DFA";

## 1. The Generic DFA Interface

To ensure our algorithms work universally across the entire pipeline (from NFA conversion to Minimization), we define a **Generic DFA interface**.

Instead of locking the state type to `number` or `string`, we use a generic type parameter `<S extends Structural>`. This allows the DFA to operate on any state representation, as long as it supports value equality:
* **Standard DFA:** States are sets of NFA states (e.g., `{1, 2, 5}`).
* **Minimized DFA:** States are sets of sets (Equivalence Classes, e.g., `{{1,3}, {2,4}}`).
* **Product DFA:** States are tuples (Pairs, e.g., `(q1, q2)`).

The implementation relies on `RecursiveSet` and `RecursiveMap` to handle these complex keys correctly.

In [ ]:
interface GenericDFA<S extends Structural> {
    Q: RecursiveSet<S>;
    Σ: RecursiveSet<Char>;
    δ: RecursiveMap<Tuple<[S, Char]>, S>;
    q0: S;
    A: RecursiveSet<S>;
}

### DFA Text Representation

The function `dfa2string` converts a generic DFA into a deterministic, human-readable text format.

**Key Challenges & Solutions:**
* **Complex States:** Since states can be deeply nested structures (like sets of sets), printing them directly in the transition table is unreadable. We alias them to simple names $S_0, S_1, \dots, S_n$.
* **Determinism:** To ensure reproducible test results, we explicitly **sort** states and alphabet characters before generating the output.
* **Legend:** To retain meaning, the output includes a legend mapping the aliases $S_i$ back to their original structural value (e.g., `S0 = {{0}, {1}}`).

In [ ]:
function dfa2string<S extends Structural>(dfa: GenericDFA<S>): string {
    const { Q, Σ, δ, q0, A } = dfa;
    let result = "";

    const stateToName = new RecursiveMap<S, string>();
    const sortedStates = [...Q].sort(RecursiveSet.compareVisual);
    
    let n = 0;
    for (const q of sortedStates) {
        stateToName.set(q, `S${n++}`);
    }

    result += `states: {S0, ..., S${n - 1}}\n\n`;

    const startName = stateToName.get(q0) ?? "UNKNOWN";
    result += `start state: ${startName}\n\n`;

    result += "state encoding:\n";
    for (const q of sortedStates) {
        result += `${stateToName.get(q)} = ${q.toString()}\n`;
    }

    result += "\ntransitions:\n";
    for (const q of sortedStates) {
        const sourceName = stateToName.get(q)!;
        const sortedSigma = [...Σ].sort();
        for (const c of sortedSigma) {
            const target = δ.get(new Tuple(q, c));

            if (target) {
                const targetName = stateToName.get(target);
                if (targetName) {
                    result += `δ(${sourceName}, ${c}) = ${targetName}\n`;
                }
            }
        }
    }

    result += "\nset of accepting states: {";
    const acceptingNames: string[] = [];

    for (const q of sortedStates) {
        if (A.has(q)) {
            acceptingNames.push(stateToName.get(q)!);
        }
    }
    result += acceptingNames.join(", ");
    result += "}\n";

    return result;
}

### DFA Visualization (DOT Format)

The function `dfa2dot` generates a **Graphviz DOT** definition to render the automaton graphically.



**Visual Encoding:**
* **Layout:** `rankdir=LR` forces a logical Left-to-Right flow.
* **Start State:** Indicated by an incoming arrow from an invisible "ghost node".
* **Accepting States:** Distinguished by a **double circle** shape.
* **Transitions:** Labeled edges representing $\delta(q, c) = p$.

Just like the text representation, states are internally aliased to $S_0...S_n$ to keep the DOT source code clean, while the visual label remains simple.

In [ ]:
function dfa2dot<S extends Structural>(dfa: GenericDFA<S>): string {
    const { Q, Σ, δ, q0, A } = dfa;
    const lines: string[] = [];

    lines.push('digraph "Deterministic FSM" {');
    lines.push('  rankdir=LR;');
    lines.push('  node [fontname="Arial", fontsize=12, shape=circle];');

    const stateToName = new RecursiveMap<S, string>();
    const sortedStates = [...Q].sort(RecursiveSet.compareVisual);

    let n = 0;
    for (const q of sortedStates) {
        stateToName.set(q, `S${n++}`);
    }

    const startName = stateToName.get(q0);
    if (startName) {
        lines.push('  "start_ghost" [label="", width=0.1, height=0.1, style=filled, color=blue];');
        lines.push(`  "start_ghost" -> "${startName}";`);
    }

    for (const q of sortedStates) {
        const name = stateToName.get(q)!;
        const shape = A.has(q) ? "doublecircle" : "circle";
        lines.push(`  "${name}" [shape="${shape}"];`);
    }

    for (const q of sortedStates) {
        const sourceName = stateToName.get(q)!;
        const sortedSigma = [...Σ].sort();

        for (const c of sortedSigma) {
            const target = δ.get(new Tuple(q, c));

            if (target) {
                const targetName = stateToName.get(target);
                if (targetName) {
                    lines.push(`  "${sourceName}" -> "${targetName}" [label="${c}"];`);
                }
            }
        }
    }

    lines.push("}");
    return lines.join("\n");
}

## 2. Rendering the Legend

When visualizing automata derived from the Powerset Construction or Minimization, state labels often become very large (nested sets). Displaying these full labels inside the graph nodes would make the diagram unreadable.

To solve this, we decouple the **structure** from the **content**:
1.  **The Graph:** Uses short integer IDs (`0`, `1`, `2`...) for the nodes.
2.  **The Legend:** An HTML table that maps these IDs back to the actual state content.

The function `renderLegend` generates this HTML table. It also visually marks the **start state** (🟢) and **accepting states** (⭐) for quick reference.

In [ ]:
function renderLegend<S extends Structural>(dfa: GenericDFA<S>): string {
    const sortedStates = [...dfa.Q].sort(RecursiveSet.compareVisual);
    
    let html = `<table style="font-family: 'Segoe UI', monospace; border-collapse: collapse; font-size: 0.85em; width: auto; margin: 10px auto; border: 1px solid #ddd; background: white; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">`;
    
    html += `<thead style="background: #f4f4f4; border-bottom: 2px solid #ccc;">
                <tr>
                    <th style="padding: 8px 12px; text-align: center; border-right: 1px solid #e0e0e0;">ID</th>
                    <th style="padding: 8px 12px; text-align: left;">State Content</th>
                    <th style="padding: 8px 12px; text-align: center; width: 40px;">Start</th>
                    <th style="padding: 8px 12px; text-align: center; width: 40px;">Final</th>
                </tr>
             </thead><tbody>`;

    let n = 0;
    for (const q of sortedStates) {
        const id = n++; // Einfache Nummerierung 0, 1, 2...
        
        const isStart = q.equals(dfa.q0) ? "🟢" : "";
        const isFinal = dfa.A.has(q) ? "⭐" : "";
        
        let content = q.toString()
            .replace(/&/g, "&amp;")
            .replace(/</g, "&lt;")
            .replace(/>/g, "&gt;");

        if (content.length > 80) {
            content = `<span style="display:inline-block; max-width: 400px; word-wrap: break-word;">${content}</span>`;
        }

        html += `<tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 6px 12px; text-align: center; font-weight: bold; color: #555; background: #fafafa; border-right: 1px solid #e0e0e0;">${id}</td>
                    
                    <td style="padding: 6px 12px; text-align: left; color: #333;">${content}</td>
                    
                    <td style="padding: 6px 12px; text-align: center;">${isStart}</td>
                    <td style="padding: 6px 12px; text-align: center;">${isFinal}</td>
                 </tr>`;
    }
    html += `</tbody></table>`;
    return html;
}

## 3. Visual Comparison Layout

To verify the equivalence of two regular expressions, it is helpful to inspect their minimized DFAs side-by-side. If the regular expressions are equivalent, the resulting graphs should be isomorphic (identical structure).

The function `renderComparisonLayout` constructs an HTML grid layout using Flexbox. It arranges the input Regular Expression, the SVG Graph, and the Legend into two columns, facilitating a direct visual comparison.

In [ ]:
function renderComparisonLayout(
    s1: string, svg1: string, legend1: string,
    s2: string, svg2: string, legend2: string
): string {
    return `
    <div style="display: flex; flex-direction: column; gap: 20px; font-family: sans-serif;">
        <div style="display: flex; gap: 20px; border: 1px solid #ccc; background: white; padding: 15px; border-radius: 5px;">
            <div style="flex: 1; min-width: 0;">
                <h4 style="margin: 0 0 10px 0; border-bottom: 2px solid #ddd; padding-bottom: 5px;">
                    RegExp 1: <code style="color: #d63384;">${s1}</code>
                </h4>
                <div style="text-align: center; margin-bottom: 10px;">${svg1}</div>
                <div style="max-height: 200px; overflow-y: auto; border: 1px solid #eee;">${legend1}</div>
            </div>
            <div style="border-left: 1px solid #ddd;"></div>
            <div style="flex: 1; min-width: 0;">
                <h4 style="margin: 0 0 10px 0; border-bottom: 2px solid #ddd; padding-bottom: 5px;">
                    RegExp 2: <code style="color: #d63384;">${s2}</code>
                </h4>
                <div style="text-align: center; margin-bottom: 10px;">${svg2}</div>
                <div style="max-height: 200px; overflow-y: auto; border: 1px solid #eee;">${legend2}</div>
            </div>
        </div>
    </div>`;
}

### NFA Text Representation

The function `nfa2string` converts a non-deterministic finite automaton (NFA) to text.

**Differences from DFA:**
* **Multiple Targets:** A transition `[q, c]` can map to a *set* of target states. We iterate through all targets.
* **Epsilon Transitions:** We specifically check for transitions labeled with $\varepsilon$ (epsilon) or empty string and list them as `[q, ""] |-> p`.

In [ ]:
function nfa2string(nfa: NFA): string {
    const { Q, Σ, δ, q0, A } = nfa;
    let result = "";

    // Sort states for deterministic output
    const sortedStates = [...Q].sort(RecursiveSet.compareVisual);

    result += `states: {${sortedStates.join(', ')}}\n\n`;
    result += `start state: ${q0}\n\n`;
    result += "transitions:\n";

    for (const q of sortedStates) {
        // 1. Explicit Transitions (Sigma)
        const sortedSigma = [...Σ].sort();
        for (const c of sortedSigma) {
            const targets = δ.get(new Tuple(q, c));
            
            if (targets && !targets.isEmpty()) {
                const sortedTargets = [...targets].sort(RecursiveSet.compareVisual);
                // Format: δ(1, a) = {2, 3}
                result += `δ(${q}, ${c}) = {${sortedTargets.join(', ')}}\n`;
            }
        }

        // 2. Epsilon Transitions
        const targetsEps = δ.get(new Tuple(q, "ε"));
        if (targetsEps && !targetsEps.isEmpty()) {
            const sortedTargets = [...targetsEps].sort(RecursiveSet.compareVisual);
            // Format: δ(1, ε) = {2}
            result += `δ(${q}, ε) = {${sortedTargets.join(', ')}}\n`;
        }
    }

    // Sort accepting states
    const sortedA = [...A].sort(RecursiveSet.compareVisual);
    result += `\nset of accepting states: {${sortedA.join(', ')}}\n`;
    return result;
}

### NFA Visualization (DOT Format)

The function `nfa2dot` visualizes an NFA.

**Visual Cues:**
* **Epsilon Edges:** Transitions labeled $\varepsilon$ are drawn with a lower edge weight (`weight="0.1"`) to encourage Graphviz to place them distinctly, often hinting at the "instant" nature of the transition.
* **Direct Mapping:** Since NFA states are typically simple primitives (numbers/strings), we use their string representation directly as node names (unlike the DFA remapping).

In [ ]:
function nfa2dot(nfa: NFA): string {
    const { Q, Σ, δ, q0, A } = nfa;
    const lines: string[] = [];

    lines.push('digraph "Non-Deterministic FSM" {');
    lines.push('  rankdir=LR;');
    lines.push('  node [fontname="Arial", fontsize=12, shape=circle];');

    lines.push('  "start_ghost" [label="", width=0.1, height=0.1, style=filled, color=blue];');
    lines.push(`  "start_ghost" -> "${q0}";`);

    const sortedStates = [...Q].sort(RecursiveSet.compareVisual);

    for (const q of sortedStates) {
        const label = String(q);
        const shape = A.has(q) ? "doublecircle" : "circle";
        lines.push(`  "${label}" [shape="${shape}"];`);
    }

    for (const q of sortedStates) {
        // Epsilon
        const targetsEps = δ.get(new Tuple(q, "ε"));
        if (targetsEps) {
            const sortedTargets = [...targetsEps].sort(RecursiveSet.compareVisual);
            for (const p of sortedTargets) {
                lines.push(`  "${q}" -> "${p}" [label="ε", weight="0.1"];`);
            }
        }

        const sortedSigma = [...Σ].sort();
        for (const c of sortedSigma) {
            const targets = δ.get(new Tuple(q, c));
            if (targets) {
                const sortedTargets = [...targets].sort(RecursiveSet.compareVisual);
                for (const p of sortedTargets) {
                    // weight="10" pulls non-epsilon transitions tighter
                    lines.push(`  "${q}" -> "${p}" [label="${c}", weight="10"];`);
                }
            }
        }
    }

    lines.push("}");
    return lines.join("\n");
}